In [34]:
import pandas as pd
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgbimport
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

# Load the Data

In [35]:
df = pd.read_csv('Export_RawData_MultiClass.csv')
df.head()

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SRC_AGENCY,FIRE_ID,FIRENAME,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,REP_DATE,ATTK_DATE,...,slope2,aspect2,LULC2,NDVI2,pcp2,soilmois2,DEM_masked,numeric_Di,numeric__1,DistMain_r
0,BC,1950-G00035,NaN,57.463,-122.816,1950,6,14,6/14/1950,18720814,...,2.11304,116.131,4.0,0.53166,615.000,58.1094,1090.0,79.9709,8.30288,137.997
1,BC,1953-G00043,NaN,57.684,-121.925,1953,5,1,5/1/1953,18720814,...,0.92424,314.215,4.0,0.51156,506.821,60.5267,801.0,88.5923,51.78940,160.362
2,BC,1950-G00113,NaN,56.789,-122.070,1950,7,29,7/29/1950,18720814,...,3.22457,113.366,4.0,0.48400,546.590,44.5410,753.0,43.6255,8.42028,72.236
3,BC,1950-G00118,NaN,56.713,-121.820,1950,7,29,7/29/1950,18720814,...,2.24412,299.632,4.0,0.46020,545.436,47.7562,906.0,40.6075,8.44729,60.000
4,BC,1950-G00133,NaN,56.789,-121.320,1950,8,26,8/26/1950,18720814,...,0.87122,260.538,2.0,0.38144,499.462,39.0829,804.0,61.6958,19.79650,58.810


In [36]:
df.columns

Index(['SRC_AGENCY', 'FIRE_ID', 'FIRENAME', 'LATITUDE', 'LONGITUDE', 'YEAR',
       'MONTH', 'DAY', 'REP_DATE', 'ATTK_DATE', 'OUT_DATE', 'DECADE',
       'SIZE_HA', 'CAUSE', 'PROTZONE', 'FIRE_TYPE', 'MORE_INFO', 'CFS_REF_ID',
       'CFS_NOTE1', 'CFS_NOTE2', 'ACQ_DATE', 'SRC_AGY2', 'ECOZONE', 'ECOZ_REF',
       'ECOZ_NAME', 'ECOZ_NOM', 'TWI2', 'temp_july2', 'slope2', 'aspect2',
       'LULC2', 'NDVI2', 'pcp2', 'soilmois2', 'DEM_masked', 'numeric_Di',
       'numeric__1', 'DistMain_r'],
      dtype='object')

In [37]:
df.shape

(18341, 38)

In [38]:
# List of columns to drop (corrected for exact names)
col_to_drop = [
    'SRC_AGENCY', 'FIRE_ID', 'FIRENAME', 'DAY', 'REP_DATE', 'ATTK_DATE', 
    'OUT_DATE', 'DECADE', 'CAUSE', 'PROTZONE', 'FIRE_TYPE', 'MORE_INFO', 'CFS_REF_ID', 
    'CFS_NOTE1', 'CFS_NOTE2', 'ACQ_DATE', 'SRC_AGY2', 'ECOZONE', 
    'ECOZ_REF', 'ECOZ_NAME', 'ECOZ_NOM'
]

# Drop the columns
df = df.drop(columns=col_to_drop, axis=1)

In [39]:
df.head()

,LATITUDE,LONGITUDE,YEAR,MONTH,SIZE_HA,TWI2,temp_july2,slope2,aspect2,LULC2,NDVI2,pcp2,soilmois2,DEM_masked,numeric_Di,numeric__1,DistMain_r
0,57.463,-122.816,1950,6,2071.900,9.98426,20.1,2.11304,116.131,4.0,0.53166,615.000,58.1094,1090.0,79.9709,8.30288,137.997
1,57.684,-121.925,1953,5,7834.700,10.81150,22.4,0.92424,314.215,4.0,0.51156,506.821,60.5267,801.0,88.5923,51.78940,160.362
2,56.789,-122.070,1950,7,99767.297,10.65960,22.2,3.22457,113.366,4.0,0.48400,546.590,44.5410,753.0,43.6255,8.42028,72.236
3,56.713,-121.820,1950,7,1035.900,9.92401,21.7,2.24412,299.632,4.0,0.46020,545.436,47.7562,906.0,40.6075,8.44729,60.000
4,56.789,-121.320,1950,8,76663.703,10.87060,22.2,0.87122,260.538,2.0,0.38144,499.462,39.0829,804.0,61.6958,19.79650,58.810


In [40]:
df['SIZE_HA'].value_counts()

200.000      264
300.000      152
500.000      117
400.000      106
1000.000     105
            ... 
23125.500      1
29197.000      1
573.500        1
5278.700       1
1250.045       1
Name: SIZE_HA, Length: 11194, dtype: int64

In [41]:
# Display rows where the 'CAUSE' column is null
null_SIZE_rows = df[df['SIZE_HA'].isnull()]
null_SIZE_rows


,LATITUDE,LONGITUDE,YEAR,MONTH,SIZE_HA,TWI2,temp_july2,slope2,aspect2,LULC2,NDVI2,pcp2,soilmois2,DEM_masked,numeric_Di,numeric__1,DistMain_r


In [43]:
df[df.isnull().any(axis= 1)]

,LATITUDE,LONGITUDE,YEAR,MONTH,SIZE_HA,TWI2,temp_july2,slope2,aspect2,LULC2,NDVI2,pcp2,soilmois2,DEM_masked,numeric_Di,numeric__1,DistMain_r


In [44]:
null_LULC_rows = df[df['LULC2'].isnull()]
null_LULC_rows


,LATITUDE,LONGITUDE,YEAR,MONTH,SIZE_HA,TWI2,temp_july2,slope2,aspect2,LULC2,NDVI2,pcp2,soilmois2,DEM_masked,numeric_Di,numeric__1,DistMain_r


In [45]:
df['LULC2'].value_counts()

 4.0       12347
 5.0        2861
 6.0         777
 11.0        754
 3.0         614
 2.0         477
 8.0         477
 9.0          23
-9999.0        7
 10.0          4
Name: LULC2, dtype: int64

In [47]:
# Filter rows where the 'LULC' column has the value -9999
lulc_invalid_rows = df[df['LULC2'] == -9999]
lulc_invalid_rows


,LATITUDE,LONGITUDE,YEAR,MONTH,SIZE_HA,TWI2,temp_july2,slope2,aspect2,LULC2,NDVI2,pcp2,soilmois2,DEM_masked,numeric_Di,numeric__1,DistMain_r
131,49.76000,-123.89400,1952,9,1388.0,-9999.000,-9999.0,-9999.00000,-9999.00000,-9999.0,-9999.00000,-9999.000,-9999.0000,384.0,218.833,83.36490,-9999.0000
11324,55.40258,-77.53582,1998,5,300.0,-9999.000,-9999.0,-9999.00000,-9999.00000,-9999.0,0.22978,643.846,22.4780,24.0,158.443,656.09200,-9999.0000
13323,55.58000,-60.62000,1997,8,259.0,-9999.000,15.5,-9999.00000,-9999.00000,-9999.0,0.20769,858.128,24.3571,19.0,167.759,616.75300,221.2230
13601,48.33000,-58.68000,1958,4,259.0,-9999.000,-9999.0,-9999.00000,-9999.00000,-9999.0,-9999.00000,-9999.000,-9999.0000,41.0,0.000,0.00000,206.7860
13647,48.39400,-54.09800,1982,7,409.0,11.685,22.2,0.38589,3.99091,-9999.0,0.51823,1088.920,38.5560,112.0,0.000,9.69265,81.6544
13668,47.40000,-53.15000,1961,6,202.0,-9999.000,-9999.0,-9999.00000,-9999.00000,-9999.0,-9999.00000,-9999.000,-9999.0000,37.0,0.000,0.00000,210.5250
13692,47.50000,-55.61000,1964,5,242.0,-9999.000,-9999.0,-9999.00000,-9999.00000,-9999.0,-9999.00000,-9999.000,-9999.0000,-9999.0,0.000,138.30000,-9999.0000


In [48]:
df.shape

(18341, 17)

In [50]:
# Drop rows where the 'LULC' column has the value -9999
df = df[df['LULC2'] != -9999]

# Display the first few rows to confirm changes
df.head()
df.shape

(18334, 17)

In [51]:
df['LULC2'].value_counts()

4.0     12347
5.0      2861
6.0       777
11.0      754
3.0       614
2.0       477
8.0       477
9.0        23
10.0        4
Name: LULC2, dtype: int64

In [52]:
replace_values_LULC = {
    1: 'Urban_areas',
    2: 'Crop_land',
    3: 'Grass_land',
    4: 'Tree_covered',
    5: 'Shrub_covered',
    6: 'Herbaceous',
    8: 'Sparse_vegetation',
    9: 'Bare_soil',
    10: 'Snow',
    11: 'Water_bodies'
}

df['LULC2'] = df['LULC2'].replace(replace_values_LULC)

C:\Users\Admin\AppData\Local\Temp/ipykernel_3660/3714413557.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LULC2'] = df['LULC2'].replace(replace_values_LULC)


In [53]:
df.head()

,LATITUDE,LONGITUDE,YEAR,MONTH,SIZE_HA,TWI2,temp_july2,slope2,aspect2,LULC2,NDVI2,pcp2,soilmois2,DEM_masked,numeric_Di,numeric__1,DistMain_r
0,57.463,-122.816,1950,6,2071.900,9.98426,20.1,2.11304,116.131,Tree_covered,0.53166,615.000,58.1094,1090.0,79.9709,8.30288,137.997
1,57.684,-121.925,1953,5,7834.700,10.81150,22.4,0.92424,314.215,Tree_covered,0.51156,506.821,60.5267,801.0,88.5923,51.78940,160.362
2,56.789,-122.070,1950,7,99767.297,10.65960,22.2,3.22457,113.366,Tree_covered,0.48400,546.590,44.5410,753.0,43.6255,8.42028,72.236
3,56.713,-121.820,1950,7,1035.900,9.92401,21.7,2.24412,299.632,Tree_covered,0.46020,545.436,47.7562,906.0,40.6075,8.44729,60.000
4,56.789,-121.320,1950,8,76663.703,10.87060,22.2,0.87122,260.538,Crop_land,0.38144,499.462,39.0829,804.0,61.6958,19.79650,58.810


In [57]:
# List of new column names
new_col_name = [
    'Latitude', 'Longitude', 'Year', 'Month', 'Size_ha', 'TWI', 'Temp_july',
    'Slope', 'Aspect', 'LULC', 'NDVI', 'Precipitation', 'SoilMoisture',
    'Elevation', 'Dist_Lakes', 'Dist_Roads', 'Dist_Rivers'
]

# Assign the new column names
df.columns = new_col_name

# Print current column names to verify they match the keys in data_types
print(df.columns)

# Dictionary mapping columns to their new data types
data_types = {
    'Latitude': float,
    'Longitude': float,
    'Year': int,
    'Month': int,
    'Size_ha': float,
    'TWI': float,
    'Temp_july': float,
    'Slope': float,
    'Aspect': float,
    'LULC': 'category',
    'NDVI': float,
    'Precipitation': float,
    'SoilMoisture': float,
    'Elevation': float,
    'Dist_Lakes': float,
    'Dist_Roads': float,
    'Dist_Rivers': float
}

# Change the data types of the columns
df = df.astype(data_types)

# Display the first few rows to confirm changes
df.head()


Index(['Latitude', 'Longitude', 'Year', 'Month', 'Size_ha', 'TWI', 'Temp_july',
       'Slope', 'Aspect', 'LULC', 'NDVI', 'Precipitation', 'SoilMoisture',
       'Elevation', 'Dist_Lakes', 'Dist_Roads', 'Dist_Rivers'],
      dtype='object')


,Latitude,Longitude,Year,Month,Size_ha,TWI,Temp_july,Slope,Aspect,LULC,NDVI,Precipitation,SoilMoisture,Elevation,Dist_Lakes,Dist_Roads,Dist_Rivers
0,57.463,-122.816,1950,6,2071.900,9.98426,20.1,2.11304,116.131,Tree_covered,0.53166,615.000,58.1094,1090.0,79.9709,8.30288,137.997
1,57.684,-121.925,1953,5,7834.700,10.81150,22.4,0.92424,314.215,Tree_covered,0.51156,506.821,60.5267,801.0,88.5923,51.78940,160.362
2,56.789,-122.070,1950,7,99767.297,10.65960,22.2,3.22457,113.366,Tree_covered,0.48400,546.590,44.5410,753.0,43.6255,8.42028,72.236
3,56.713,-121.820,1950,7,1035.900,9.92401,21.7,2.24412,299.632,Tree_covered,0.46020,545.436,47.7562,906.0,40.6075,8.44729,60.000
4,56.789,-121.320,1950,8,76663.703,10.87060,22.2,0.87122,260.538,Crop_land,0.38144,499.462,39.0829,804.0,61.6958,19.79650,58.810


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18334 entries, 0 to 18340
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Latitude       18334 non-null  float64 
 1   Longitude      18334 non-null  float64 
 2   Year           18334 non-null  int32   
 3   Month          18334 non-null  int32   
 4   Size_ha        18334 non-null  float64 
 5   TWI            18334 non-null  float64 
 6   Temp_july      18334 non-null  float64 
 7   Slope          18334 non-null  float64 
 8   Aspect         18334 non-null  float64 
 9   LULC           18334 non-null  category
 10  NDVI           18334 non-null  float64 
 11  Precipitation  18334 non-null  float64 
 12  SoilMoisture   18334 non-null  float64 
 13  Elevation      18334 non-null  float64 
 14  Dist_Lakes     18334 non-null  float64 
 15  Dist_Roads     18334 non-null  float64 
 16  Dist_Rivers    18334 non-null  float64 
dtypes: category(1), float64(14), in

## Multi-Collinearity

In [60]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgb
import matplotlib.pyplot as plt

def calculate_vif(df, exclude_columns=None):
    # Set default for exclude_columns if not provided
    if exclude_columns is None:
        exclude_columns = []

    # Always exclude 'Wildfire' in addition to other specified columns
    always_exclude = ['Wildfire', 'Year', 'Latitude', 'Longitude', 'Month']
    all_exclusions = set(exclude_columns).union(always_exclude)

    # Drop excluded columns
    df_filtered = df.drop(columns=all_exclusions, errors='ignore')  # 'errors=ignore' prevents errors if the column doesn't exist

    # Convert categorical columns to numerical codes
    df_filtered = df_filtered.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)

    # Calculate VIF for each variable that remains
    vifs = [variance_inflation_factor(df_filtered.values, i) for i in range(df_filtered.shape[1])]
    tolerance = [1/vif for vif in vifs]

    # Prepare the VIF DataFrame
    vif_data = pd.DataFrame({
        'VIF': vifs,
        'Tolerance': tolerance
    }, index=df_filtered.columns)

    return vif_data

# Specify additional columns to exclude as needed
additional_excludes = []

# Calculate VIF excluding specified columns
vif_data = calculate_vif(df, exclude_columns=additional_excludes)

print(vif_data)


                     VIF  Tolerance
Size_ha         1.077603   0.927986
TWI            18.217779   0.054891
Temp_july       3.118036   0.320715
Slope           2.470375   0.404797
Aspect          3.816462   0.262023
LULC           11.960936   0.083605
NDVI           84.451106   0.011841
Precipitation   7.617432   0.131278
SoilMoisture   92.509191   0.010810
Elevation       4.651523   0.214983
Dist_Lakes      2.702171   0.370073
Dist_Roads      3.106148   0.321942
Dist_Rivers     2.727467   0.366640


In [64]:
df.shape

(18334, 17)

In [65]:
df.head()

,Latitude,Longitude,Year,Month,Size_ha,TWI,Temp_july,Slope,Aspect,LULC,NDVI,Precipitation,SoilMoisture,Elevation,Dist_Lakes,Dist_Roads,Dist_Rivers
0,57.463,-122.816,1950,6,2071.900,9.98426,20.1,2.11304,116.131,Tree_covered,0.53166,615.000,58.1094,1090.0,79.9709,8.30288,137.997
1,57.684,-121.925,1953,5,7834.700,10.81150,22.4,0.92424,314.215,Tree_covered,0.51156,506.821,60.5267,801.0,88.5923,51.78940,160.362
2,56.789,-122.070,1950,7,99767.297,10.65960,22.2,3.22457,113.366,Tree_covered,0.48400,546.590,44.5410,753.0,43.6255,8.42028,72.236
3,56.713,-121.820,1950,7,1035.900,9.92401,21.7,2.24412,299.632,Tree_covered,0.46020,545.436,47.7562,906.0,40.6075,8.44729,60.000
4,56.789,-121.320,1950,8,76663.703,10.87060,22.2,0.87122,260.538,Crop_land,0.38144,499.462,39.0829,804.0,61.6958,19.79650,58.810


In [72]:
df['Size_ha'].describe()

count    1.833400e+04
mean     7.060997e+03
std      2.597154e+04
min      2.000000e+02
25%      4.370000e+02
50%      1.168145e+03
75%      4.384095e+03
max      1.050000e+06
Name: Size_ha, dtype: float64

In [73]:
# Define a function to categorize wildfire sizes
def categorize_wildfire_size(size):
    if size < 1000:
        return 'Low'
    elif 1000 <= size < 10000:
        return 'Moderate'
    elif 10000 <= size < 50000:
        return 'High'
    else:
        return 'Extreme'

# Apply the function to create a new column 'Size_Category'
df['WF_class'] = df['Size_ha'].apply(categorize_wildfire_size)

# Display the first few rows to verify
print(df.head())

# Optional: Display the counts for each category
category_counts = df['WF_class'].value_counts()
print(category_counts)


   Latitude  Longitude  Year  Month    Size_ha       TWI  Temp_july    Slope  \
0    57.463   -122.816  1950      6   2071.900   9.98426       20.1  2.11304   
1    57.684   -121.925  1953      5   7834.700  10.81150       22.4  0.92424   
2    56.789   -122.070  1950      7  99767.297  10.65960       22.2  3.22457   
3    56.713   -121.820  1950      7   1035.900   9.92401       21.7  2.24412   
4    56.789   -121.320  1950      8  76663.703  10.87060       22.2  0.87122   

    Aspect          LULC     NDVI  Precipitation  SoilMoisture  Elevation  \
0  116.131  Tree_covered  0.53166        615.000       58.1094     1090.0   
1  314.215  Tree_covered  0.51156        506.821       60.5267      801.0   
2  113.366  Tree_covered  0.48400        546.590       44.5410      753.0   
3  299.632  Tree_covered  0.46020        545.436       47.7562      906.0   
4  260.538     Crop_land  0.38144        499.462       39.0829      804.0   

   Dist_Lakes  Dist_Roads  Dist_Rivers  WF_class  
0    